1 - Imports

In [1]:
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
from collections import Counter

2 - We extract the data from the file

In [2]:
def extract_data(message_file):
    # Create dataframe
    data = pd.DataFrame(columns=['Message', 'Spam'])

    # Open the file
    with open(message_file, 'r') as file:
        for line in file:
            # Get spam boolean value and line string
            if line.split()[0] == 'ham':
                spam = 0
                line = line[4:-1]
            elif line.split()[0] == 'spam':
                spam = 1
                line = line[8:-1]
            
            # Append new row
            data = data.append({ 'Message': line, 'Spam': spam }, ignore_index=True)
    
    return data

data = extract_data('messages.txt')
data.head()

Message Spam
0                    Yup i've finished c Ã¼ there...    0
1               Remember to ask alex about his pizza    0
2                       No da..today also i forgot..    0
3  Ola would get back to you maybe not today but ...    0
4  Fwiw the reason I'm only around when it's time...    0

3 - Divide the data in training and test examples

In [3]:
def divide_data(data, train_frac=0.8):
    # Divide
    df_train = data.sample(frac=train_frac)
    df_test = data.drop(df_train.index)
    
    # Reset indexes
    df_train = df_train.reset_index()
    df_train.drop(columns='index', inplace=True)
    df_test = df_test.reset_index()
    df_test.drop(columns='index', inplace=True)
    
    return df_train, df_test

df_train, df_test = divide_data(data)

print(df_train.shape)
print(df_test.shape)

(4000, 2)
(1000, 2)


4 - Generate a dictionary from the training data

In [4]:
def make_dictionary(df, nb_words=3000):
    all_words = []
    
    # Read all the data
    for i, row in df.iterrows():
        # Add words
        all_words += row['Message'].split()
        
    # Lowercase all the words
    all_words = [word.lower() for word in all_words]
    
    # Create dictionary
    dictionary = Counter(all_words)
    
    # Remove bad items
    for item in list(dictionary):
        # Non-alphanumeric words
        if item.isalpha() == False:
            del dictionary[item]
        # One letter words
        elif len(item) == 1:
            del dictionary[item]
    
    # Keep the *nb_words* most common words
    dictionary = dictionary.most_common(nb_words)
    
    return dictionary
              
dictionary = make_dictionary(df_train)
print(dictionary[:4])

[('to', 1579), ('you', 1339), ('the', 940), ('and', 693)]


5 - Extract features from the data sets

In [11]:
def extract_features(df, dictionary, multi=True):
    # Add feature columns to dataframe
    for word_id, d in enumerate(dictionary):
        df[d[0]] = 0
    
    # Read all rows
    for row_id, row in df.iterrows():
        # Get all the words of the message
        word_list = row['Message'].split()
        
        # For all these words
        for word in word_list:
            # For all the words in the dictionary
            for word_id, d in enumerate(dictionary):
                if d[0] == word.lower():
                    # Add the count of this word in the right columns
                    df.at[row_id, d[0]] = word_list.count(word)
    
    # Remove 'Message' column
    df.drop(columns='Message', inplace=True)

extract_features(df_train, dictionary)
extract_features(df_test, dictionary)
df_train.head()

Spam  to  you  the  and  is  in  my  for  your    ...     coccooning  \
0    1   0    0    0    0   0   0   0    0     1    ...              0   
1    0   0    0    0    0   0   0   1    0     0    ...              0   
2    0   0    1    0    2   0   1   0    0     1    ...              0   
3    0   0    2    0    0   0   0   0    0     0    ...              0   
4    0   0    0    0    0   0   0   0    0     0    ...              0   

   pattern  url  entertaining  hugh  laurie  stick  resizing  dying  settling  
0        0    0             0     0       0      0         0      0         0  
1        0    0             0     0       0      0         0      0         0  
2        0    0             0     0       0      0         0      0         0  
3        0    0             0     0       0      0         0      0         0  
4        0    0             0     0       0      0         0      0         0  

[5 rows x 3001 columns]

6 - Implement the Naive Bayes algorithm